In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy import stats
import requests
import gmaps
import os
import json
from opencage.geocoder import OpenCageGeocode

In [3]:
#Load FEMA Data Sets
owner_data = pd.read_csv('interim_data/owner_export_clean.csv', encoding='cp1252')
renters_data = pd.read_csv('interim_data/renter_export_clean.csv')

In [4]:
#Visualize Dataframes
owner_data.head(5)

,Disaster,State,County,City,Zip Code,Valid Registrations,Average FEMA Inspected Damage,Total Inspected,Total Damage,No FEMA Inspected Damage,...,"FEMA Inspected Damage > $30,000",Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4447.0,OH,Auglaize (County),CRIDERSVILLE,45806.0,1,$-,1.0,$-,1.0,...,0,0,$0,$0,$0,$0,0,0,0,0
1,4447.0,OH,Auglaize (County),NEW KNOXVILLE,45871.0,1,"$2,376.23",1.0,"$2,376.23",0.0,...,0,1,$697.52,$698,$0,$0,1,0,0,0
2,4447.0,OH,Auglaize (County),SAINT MARYS,45885.0,3,$-,3.0,$-,3.0,...,0,0,$0,$0,$0,$0,0,0,0,0
3,4447.0,OH,Auglaize (County),WAPAKONETA,45895.0,21,"$6,735.89",14.0,"$94,302.44",4.0,...,1,5,"$56,475.44","$41,315","$2,406","$12,754",3,1,1,0
4,4447.0,OH,Darke (County),ARCANUM,45304.0,1,$-,1.0,$-,1.0,...,0,0,$0,$0,$0,$0,0,0,0,0


In [5]:
#Visualize Dataframes
renters_data.head(5)

,Disaster,State,County,City,Zip Code,Valid Registrations,Total Inspected,Inspected with No Damage,Total with Moderate Damage,Total with Major Damage,Total with Substantial Damage,Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4454,TX,Cameron (County),BROWNSVILLE,78521,2,0,0,0,0,0,0,$-,$-,$-,$0,0,0,0,0
1,4454,TX,Cameron (County),BROWNSVILLE,78526,1,1,1,0,0,0,0,$-,$-,$-,$0,0,0,0,0
2,4454,TX,Cameron (County),CAMERON,76520,1,1,1,0,0,0,0,$-,$-,$-,$0,0,0,0,0
3,4454,TX,Cameron (County),COMBES,78535,7,7,6,1,0,0,2,"$4,197.14",$-,"$1,436.00","$2,761",2,0,0,0
4,4454,TX,Cameron (County),HARLINGEN,78550,224,179,110,69,0,0,94,"$227,526.33",$-,"$102,614.12","$124,912",94,0,0,0


In [6]:
#Make copy and clean data sets
converted_renters_data = renters_data.copy()
converted_renters_data['Total Approved IHP Amount'] = converted_renters_data['Total Approved IHP Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_renters_data['Rental Amount'] = converted_renters_data['Rental Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_renters_data['Repair/Replace Amount'] = converted_renters_data['Repair/Replace Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_renters_data['Other Needs Amount'] = converted_renters_data['Other Needs Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')




In [7]:
converted_renters_data.head(5)

,Disaster,State,County,City,Zip Code,Valid Registrations,Total Inspected,Inspected with No Damage,Total with Moderate Damage,Total with Major Damage,Total with Substantial Damage,Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4454,TX,Cameron (County),BROWNSVILLE,78521,2,0,0,0,0,0,0,0.00,0.0,0.00,0.0,0,0,0,0
1,4454,TX,Cameron (County),BROWNSVILLE,78526,1,1,1,0,0,0,0,0.00,0.0,0.00,0.0,0,0,0,0
2,4454,TX,Cameron (County),CAMERON,76520,1,1,1,0,0,0,0,0.00,0.0,0.00,0.0,0,0,0,0
3,4454,TX,Cameron (County),COMBES,78535,7,7,6,1,0,0,2,4197.14,0.0,1436.00,2761.0,2,0,0,0
4,4454,TX,Cameron (County),HARLINGEN,78550,224,179,110,69,0,0,94,227526.33,0.0,102614.12,124912.0,94,0,0,0


In [8]:
#Makes copy and clean data set
converted_owner_data = owner_data.copy()
converted_owner_data["Total Approved IHP Amount"] = converted_owner_data["Total Approved IHP Amount"].str.replace(',','').str.replace('$','').astype('float')
converted_owner_data['Average FEMA Inspected Damage'] = converted_owner_data["Average FEMA Inspected Damage"].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Rental Amount'] = converted_owner_data['Rental Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Repair/Replace Amount'] = converted_owner_data['Repair/Replace Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Other Needs Amount'] = converted_owner_data['Other Needs Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Approved between $1 and $10,000'] = converted_owner_data['Approved between $1 and $10,000'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Approved between $10,001 and $25,000'] = converted_owner_data['Approved between $10,001 and $25,000'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Approved between $25,001 and Max'] = converted_owner_data['Approved between $25,001 and Max'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Total Max Grants'] = converted_owner_data['Total Max Grants'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Total Damage'] = converted_owner_data['Total Damage'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')


In [9]:
converted_owner_data.head(5)

,Disaster,State,County,City,Zip Code,Valid Registrations,Average FEMA Inspected Damage,Total Inspected,Total Damage,No FEMA Inspected Damage,...,"FEMA Inspected Damage > $30,000",Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4447.0,OH,Auglaize (County),CRIDERSVILLE,45806.0,1,0.00,1.0,0.00,1.0,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4447.0,OH,Auglaize (County),NEW KNOXVILLE,45871.0,1,2376.23,1.0,2376.23,0.0,...,0,1,697.52,698.0,0.0,0.0,1.0,0.0,0.0,0.0
2,4447.0,OH,Auglaize (County),SAINT MARYS,45885.0,3,0.00,3.0,0.00,3.0,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4447.0,OH,Auglaize (County),WAPAKONETA,45895.0,21,6735.89,14.0,94302.44,4.0,...,1,5,56475.44,41315.0,2406.0,12754.0,3.0,1.0,1.0,0.0
4,4447.0,OH,Darke (County),ARCANUM,45304.0,1,0.00,1.0,0.00,1.0,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#Count the number of inputs by state
state_counts_rent = converted_renters_data["State"].value_counts()
state_counts_rent.head(5)

FL    5774
PR    4638
TX    3287
NC    1385
MP     999
Name: State, dtype: int64

In [11]:
#Count the number of inputs by state
state_counts_owners = converted_owner_data["State"].value_counts()
state_counts_owners.head(5)

PR    7649
FL    4580
TX    2660
NC    1483
LA    1147
Name: State, dtype: int64

In [12]:
# Using GroupBy in order to separate the data into fields according to "state" values
grouped_renters_df = converted_renters_data.groupby(['State'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_renters_df)

# In order to be visualized, a data function must be used...
grouped_renters_df.count().head(5)

,Disaster,County,City,Zip Code,Valid Registrations,Total Inspected,Inspected with No Damage,Total with Moderate Damage,Total with Major Damage,Total with Substantial Damage,Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
State,,,,,,,,,,,,,,,,,,,
AK,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40
AL,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162
AR,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259
AS,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
CA,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619


In [13]:
# Using GroupBy in order to separate the data into fields according to "state" values
grouped_owners_df = converted_owner_data.groupby(['State'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_owners_df)

# In order to be visualized, a data function must be used...
grouped_owners_df.count().head(5)

,Disaster,County,City,Zip Code,Valid Registrations,Average FEMA Inspected Damage,Total Inspected,Total Damage,No FEMA Inspected Damage,"FEMA Inspected Damage between $1 and $10,000",...,"FEMA Inspected Damage > $30,000",Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
State,,,,,,,,,,,,,,,,,,,,,
AL,33,33,33,33,33,33,33,33,33,33,...,33,33,33,33,33,33,33,33,33,33
AR,232,232,232,232,232,232,232,232,232,232,...,232,232,232,232,232,232,232,232,232,232
AS,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
CA,461,461,461,461,461,461,461,461,461,461,...,461,461,461,461,461,461,461,461,461,461
FL,4580,4580,4580,4580,4580,4580,4580,4580,4580,4580,...,4580,4580,4580,4580,4580,4580,4580,4580,4580,4580


In [14]:
#Create a dataframe
count = grouped_owners_df['Disaster'].count()
total = grouped_owners_df['Total Approved IHP Amount'].sum()
grouped_owners_df['Total Approved IHP Amount'].sum()
state_summary_table_owners = pd.DataFrame({"Number of Claims": count,
                                    "Total Approved IHP Amount": total})

In [15]:
#Show the dataframe
state_summary_table_owners.head(5)

,Number of Claims,Total Approved IHP Amount
State,,
AL,33,4.068319e+05
AR,232,4.328960e+06
AS,19,1.860799e+07
CA,461,1.646580e+07
FL,4580,5.486111e+08


In [16]:
#Create new datafram
count1 = grouped_renters_df['Disaster'].count()
total1 = grouped_renters_df['Total Approved IHP Amount'].sum()
grouped_renters_df['Total Approved IHP Amount'].sum()
state_summary_table_renters = pd.DataFrame({"Number of Claims": count1,
                                    "Total Approved IHP Amount": total1})

In [17]:
#Prep for merger
state_summary_table_combined = state_summary_table_renters.copy()

In [18]:
#Combine two dfs renters and owners
state_summary_table_combined ['Number of Claims Owners'] = state_summary_table_owners['Number of Claims']
state_summary_table_combined ['Total Approved IHP Amount Owners'] = state_summary_table_owners['Total Approved IHP Amount']
state_summary_table_combined ['Number of Claims'] = state_summary_table_owners['Number of Claims'] + state_summary_table_combined ['Number of Claims Owners']
state_summary_table_combined ['Total Approved IHP Amount'] = state_summary_table_owners['Total Approved IHP Amount'] + state_summary_table_combined ['Total Approved IHP Amount Owners']
state_summary_table_combined = state_summary_table_combined.drop(['Number of Claims Owners','Total Approved IHP Amount Owners'], axis=1)

In [19]:
#Show merged table
state_summary_table_combined.head()

,Number of Claims,Total Approved IHP Amount
State,,
AK,NaN,NaN
AL,66.0,813663.80
AR,464.0,8657919.20
AS,38.0,37215975.34
CA,922.0,32931609.72


In [20]:
#Create combined df and groupby state and couhty
converted_owner_data_slimed = converted_owner_data[['State','County','Total Approved IHP Amount']]
converted_renters_data_slimed = converted_owner_data[['State','County','Total Approved IHP Amount']]
converted_combined_data_slimed = pd.merge(converted_owner_data_slimed, converted_renters_data_slimed, how='outer' )
converted_combined_data_slimed = converted_combined_data_slimed.drop_duplicates(subset=None, keep='first', inplace=False)
county_summary_table = converted_combined_data_slimed.groupby(['State','County']).sum()

In [23]:
#Visualize df
county_summary_table


Total Approved IHP Amount
State County                                        
AL    Calhoun (County)                     181921.28
      Cullman (County)                     132120.56
      Etowah (County)                       92790.06
AR    Benton (County)                      354116.72
      Boone (County)                        84391.52
...                                              ...
WV    Pocahontas (County)                  175721.87
      Roane (County)                      1561434.38
      Summers (County)                     376390.06
      Webster (County)                    2405660.11
      Wetzel (County)                      887050.49

[566 rows x 1 columns]

In [24]:
#Reset index so we can use state in loop
county_summary_table_a = county_summary_table.reset_index()
a = county_summary_table_a
a

,State,County,Total Approved IHP Amount
0,AL,Calhoun (County),181921.28
1,AL,Cullman (County),132120.56
2,AL,Etowah (County),92790.06
3,AR,Benton (County),354116.72
4,AR,Boone (County),84391.52
...,...,...,...
561,WV,Pocahontas (County),175721.87
562,WV,Roane (County),1561434.38
563,WV,Summers (County),376390.06
564,WV,Webster (County),2405660.11


In [25]:
#Enter Open Cage key and import it into function
key = ""
geocoder = OpenCageGeocode(key)

In [26]:
#Build loop to request lat and long from Geocoder

#List to append info into
lat = []
lng = []
query = []

#loops through data frame takes county and state and request info
for index, row in a.iterrows():
    if row['State'] == 'PR':
        query = "u'" + row['County'] + ", " + row['State'] + " ,PRI'"
        results = geocoder.geocode(query)
        #Saves lat and long into list
        lat.append(results[0]['geometry']['lat']) 
        lng.append(results[0]['geometry']['lng'])
    else:
        query = "u'" + row['County'] + ", " + row['State'] + " ,US'"
        results = geocoder.geocode(query)
        #Saves lat and long into list
        lat.append(results[0]['geometry']['lat']) 
        lng.append(results[0]['geometry']['lng'])
    


NotAuthorizedError: Your API key is not authorized. You may have entered it incorrectly.

In [ ]:
#Enter latitude into df
county_summary_table_a['lat'] = lat

In [ ]:
#Enter longitude into df
county_summary_table_a['lng'] = lng

In [ ]:
county_summary_table_a.head(10)

In [ ]:
county_summary_table_aid = county_summary_table_a.sort_values(['Total Approved IHP Amount'], ascending=False)
county_summary_table_aid['Total Approved IHP Amount'] = county_summary_table_aid['Total Approved IHP Amount'].map('${:,.2f}'.format)
county_summary_table_aid.head(10)

In [ ]:
#Enter Google Geocode Key
gkey = ""

In [ ]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [ ]:
#Save lat and lng as a dataframe
locations = county_summary_table_a[['lat','lng']].astype(float)
#Store humidity as array
approved_IHP_amount = county_summary_table_a['Total Approved IHP Amount'].astype(float)
#Set Humidity as weight
max_approved_IHP_amount = approved_IHP_amount.max()

In [ ]:
#Customize the size of the figure
figure_layout = {
    'width': '700px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


In [ ]:
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights = approved_IHP_amount, dissipating=False, max_intensity=max_approved_IHP_amount, point_radius=3)
fig.add_layer(heat_layer)
fig